In [14]:
%matplotlib inline

from osgeo import gdal
import rasterio
from matplotlib import pyplot
import numpy
import pygeoprocessing

# Landsat 8 datasets from June 5, 2013
L8_RED_2013 = '../data/landsat/LC08_L1TP_042034_20130605_20170310_01_T1_B4_120x120.TIF'
L8_NIR_2013 = '../data/landsat/LC08_L1TP_042034_20130605_20170310_01_T1_B5_120x120.TIF'
L8_QA_2013 = '../data/landsat/LC08_L1TP_042034_20130605_20170310_01_T1_BQA_120x120.TIF'
L8_2013_FILES = [L8_RED_2013, L8_NIR_2013, L8_QA_2013]

# Landsat 8 datasets from June 29, 2016
L8_RED_2016 = '../data/landsat/LC08_L1TP_042034_20160629_20170222_01_T1_B4_120x120.TIF'
L8_NIR_2016 = '../data/landsat/LC08_L1TP_042034_20160629_20170222_01_T1_B5_120x120.TIF'
L8_QA_2016 = '../data/landsat/LC08_L1TP_042034_20160629_20170222_01_T1_BQA_120x120.TIF'
L8_2016_FILES = [L8_RED_2016, L8_NIR_2016, L8_QA_2016]

# We'll define an output nodata value.
NODATA = -9999

def plot(array):
    """Plot a numpy array with an NDVI colormap."""
    pyplot.imshow(array, cmap='RdYlGn')
    pyplot.colorbar()
    
    
def calc_ndvi(red, nir, qa):
    """Calculate NDVI from red and near-infrared landsat bands."""
    red = red.astype(numpy.float32)
    nir = nir.astype(numpy.float32)
    
    ndvi = (nir - red) / (nir + red)
    ndvi[qa==1] = NODATA
    return ndvi

# Let's calculate change in NDVI between 2013 and 2016!